In [ ]:
# pip install pysam

In [ ]:
# get reads

import pysam

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1")
for index, x in enumerate(iter):
    if index>=1:
        break
    print(x)
    print('-----------------------')
    print('chr:',x.reference_name)
    print('pos:',x.reference_start)
    print('mate chr:',x.next_reference_name)
    print('mate pos:',x.next_reference_start)
    print('read length:',x.query_alignment_length)

In [ ]:
# find sample stats
import pysam
import statistics 

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
numberReads = 0
insertSizeList = []
for x in iter:
    numberReads += 1
    insertsize = x.next_reference_start - x.reference_start + x.query_alignment_length
    if x.is_read1 and insertsize>0 and x.next_reference_start-x.reference_start<2000 and x.next_reference_name==x.reference_name:
        insertSizeList.append(x.next_reference_start-x.reference_start)

mean = statistics.mean(insertSizeList)
stdev = statistics.stdev(insertSizeList)
print("mean :", mean)
print("stdev :", stdev)
print("number of reads :", numberReads)

In [ ]:
# find deletion

import pysam

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
numberReads = 0
detectPoslast = 0
for x in iter:
    insertsize = x.next_reference_start - x.reference_start + x.query_alignment_length
    if x.is_read1 and insertsize>mean+(stdev*5) and x.next_reference_name==x.reference_name:
        if x.reference_start>=detectPoslast+4000:
            detectPoslast = x.reference_start
            print('------------------------------')
        print('pos:',x.reference_start,' mate pos:',x.next_reference_start,' insertsize:',insertsize)


In [ ]:
# find insertion

import pysam

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
detectPoslast = 0
for x in iter:
    if x.mate_is_unmapped:
        if x.reference_start>=detectPoslast+4000:
            detectPoslast = x.reference_start
            print('------------------------------')
        print('pos:',x.reference_start)
       


In [ ]:
# find tandem duplicatoion by rc

import pysam
import math
import statistics 

length = 10000
def roundup(x):
    return int(math.ceil(x / length)) * length

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
d = {}
for x in iter:
    if d.get(roundup(x.reference_start)) is not None:
        d[roundup(x.reference_start)] += 1
    else:
        d[roundup(x.reference_start)] = 0

depthlist = []
for k,v in d.items():
    depthlist.append(v)

mean = statistics.mean(depthlist)
stdev = statistics.stdev(depthlist)
print("mean :", mean)
print("stdev :", stdev)

for k,v in d.items():
    print(k, ' => ', v)


In [ ]:
# find tandem duplicatoion by rc

import pysam
import math
import statistics 

length = 10000
def roundup(x):
    return int(math.ceil(x / length)) * length

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
numberReads = 0
d = {}
for x in iter:
    if d.get(roundup(x.reference_start)) is not None:
        d[roundup(x.reference_start)] += 1
    else:
        d[roundup(x.reference_start)] = 0

depthlist = []
for k,v in d.items():
    depthlist.append(v)

mean = statistics.mean(depthlist)
stdev = statistics.stdev(depthlist)
print("mean :", mean)
print("stdev :", stdev)

for k,v in d.items():
    if v> mean+(stdev*2):
        print(k, ' => ', v)

In [ ]:
# find deletion by RC

import pysam
import math
import statistics 

length = 10000
def roundup(x):
    return int(math.ceil(x / length)) * length

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
d = {}
for x in iter:
    if d.get(roundup(x.reference_start)) is not None:
        d[roundup(x.reference_start)] += 1
    else:
        d[roundup(x.reference_start)] = 0

depthlist = []
for k,v in d.items():
    depthlist.append(v)

mean = statistics.mean(depthlist)
stdev = statistics.stdev(depthlist)
print("mean :", mean)
print("stdev :", stdev)

for k,v in d.items():
    if v < mean-(stdev*2):
        print(k, ' => ', v)

In [ ]:
# find translocation

import pysam

samfile = pysam.AlignmentFile("sample.bam", "rb")
iter = samfile.fetch("chr1", 60000000, 70000000)
for x in iter:
    if x.next_reference_name!=x.reference_name:
        print('chr:',x.reference_name,' pos:',x.reference_start,' mate chr:',x.next_reference_name,' mate pos:',x.next_reference_start)